In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import sys
import time

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [4]:
from data.datasets.h36m import Human36M
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

from experiments.common import exp_init

In [5]:
from model import blocks
from model import layers
from model import losses
from model import config
from model import callbacks
from model.utils import pose_format, log

In [6]:
from model.networks.multi_branch_model import MultiBranchModel
from model.networks.mbm_vgg import MultiBranchVGGModel

In [7]:
from model.networks.mbm_reduced import MultiBranchReduced

# Dataset

In [8]:
# local loading
h36m_path = '/home/caleml/datasets/h36m'
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [ ]:
conf = {
    'exp_type': 'hybrid_vgg_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [9]:
conf = {
    'exp_type': 'hybrid_reduced_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [13]:
model_folder = exp_init(conf)

Conducting experiment for 60 epochs and 2 blocks in folder /home/caleml/pe_experiments/exp_20190429_1233_hybrid_reduced_TEST_NB__2b_bs8


In [ ]:
# validation dataset
h36m_val = BatchLoader(
    h36m, 
    ['frame'], 
    ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
    VALID_MODE, 
    batch_size=h36m.get_length(VALID_MODE), 
    shuffle=True)

In [ ]:
log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')
[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

In [ ]:
eval_callback = callbacks.H36MEvalCallback(conf['pose_blocks'], x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, logdir=model_folder)

# Training

In [ ]:
# VGG with action for phony placeholders
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['action'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [10]:
# VGG loader with phony keyword
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

y_dict ['phony', 'phony', 'phony', 'pose', 'pose'], allkeys ['frame', 'pose', 'pose']


In [ ]:
# VGG loader with specific phony sizes
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony_2_b_256_256_64', 'phony_2_b_128_128_128', 'phony_2_b_64_64_256'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [15]:
# classical i_hat + pose format 
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['frame'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

y_dict ['frame', 'pose', 'pose'], allkeys ['frame', 'frame', 'pose', 'pose']


[autoreload of model.networks.multi_branch_model failed: Traceback (most recent call last):
  File "/usr/lib/python3.4/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/lib/python3.4/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/usr/lib64/python3.4/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib64/python3.4/importlib/__init__.py", line 149, in reload
    methods.exec(module)
  File "<frozen importlib._bootstrap>", line 1153, in exec
  File "<frozen importlib._bootstrap>", line 1129, in _exec
  File "<frozen importlib._bootstrap>", line 1467, in exec_module
  File "<frozen importlib._bootstrap>", line 1572, in get_code
  File "<frozen importlib._bootstrap>", line 1532, in source_to_code
  File "<frozen importlib._bootstrap>", line 321, in _call_with_frames_removed
  File "/home/caleml/main-pe/model/networks/multi_b

### Classical multi branch

In [ ]:
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
# model.add_callback(eval_callback)

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

In [ ]:
# short test for cb
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'])

### VGG multi branch

In [ ]:
model = MultiBranchVGGModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

### Reduced multi branch

In [12]:
model = MultiBranchReduced(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
full ResNet18 model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 128, 128, 64) 9472        input_17[0][0]                   
__________________________________________________________________________________________________
batch_normalization_47 (BatchNo (None, 128, 128, 64) 256         conv2d_46[0][0]                  
__________________________________________________________________________________________________
activation_46 (Activation)      (None, 128, 128, 64) 0  

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 128, 128, 64) 9472        input_17[0][0]                   
__________________________________________________________________________________________________
batch_normalization_47 (BatchNo (None, 128, 128, 64) 256         conv2d_46[0][0]                  
__________________________________________________________________________________________________
activation_46 (Activation)      (None, 128, 128, 64) 0           batch_normalization_47[0][0]     
__________________________________________________________________________________________________
max_poolin

pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
Last H shape Tensor("fReMap2_1/batch_normalization_84/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
Build E_a 1.6559102535247803, build E_p 12.058573246002197, decoder D 0.5415608882904053
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 128), shape z_p (?, 16, 16, 128)
Outputs shape [(None, 256, 256, 3), (None, 17, 4), (None, 17, 4)]
rec y_pred shape (?, 256, 256, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_6 (Model)                 (None, 16, 16, 128)  690368      input_32[0][0]      

In [14]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

Training with 0 callbacks
Epoch 1/60


ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 5 arrays: [array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]]), array([[ True],
       [ True],
       [ True],
       [ True],
     ...

# VGG debug stuff

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [ ]:
vgg_model.summary()

In [ ]:
output_layers = [1,3,4,6,7]
for i in output_layers:
    print(vgg_model.layers[i].name)

In [ ]:
# conv1_1,conv2_1,conv3_1,pool1,pool2
# from https://discuss.pytorch.org/t/how-to-use-vgg-19-network-to-estimate-perceptual-loss/9981

In [ ]:
a = 'phony_2_b_256_256_64'
batch_size = 24
b = [int(elt) if elt.lower() != 'b' else batch_size for elt in a.split('_')[1:]]

In [ ]:
b